In [ ]:
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import seaborn as sns
from scipy.signal import periodogram,welch
from scipy import signal as sig
from scipy.integrate import simps
from sklearn.neighbors import KDTree
from math import factorial, log
import math
from decimal import Decimal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def spectral_entropy(x, sf, method='fft', nperseg=None, normalize=False,
                     axis=-1):
    x = np.asarray(x)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf, axis=axis)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
    if normalize:
        se /= np.log2(psd_norm.shape[axis])
    return se

In [ ]:
def _embed(x, order=3, delay=1):
    """Time-delay embedding.

    Parameters
    ----------
    x : 1d-array
        Time series, of shape (n_times)
    order : int
        Embedding dimension (order).
    delay : int
        Delay.

    Returns
    -------
    embedded : ndarray
        Embedded time-series, of shape (n_times - (order - 1) * delay, order)
    """
    N = len(x)
    if order * delay > N:
        raise ValueError("Error: order * delay should be lower than x.size")
    if delay < 1:
        raise ValueError("Delay has to be at least 1.")
    if order < 2:
        raise ValueError("Order has to be at least 2.")
    Y = np.zeros((order, N - (order - 1) * delay))
    for i in range(order):
        Y[i] = x[(i * delay):(i * delay + Y.shape[1])]
    return Y.T



def _app_samp_entropy(x, order, metric='chebyshev', approximate=True):
    """Utility function for `app_entropy`` and `sample_entropy`.
    """
    _all_metrics = KDTree.valid_metrics
    if metric not in _all_metrics:
        raise ValueError('The given metric (%s) is not valid. The valid '
                         'metric names are: %s' % (metric, _all_metrics))
    phi = np.zeros(2)
    r = 0.2 * np.std(x, ddof=0)

    # compute phi(order, r)
    _emb_data1 = _embed(x, order, 1)
    if approximate:
        emb_data1 = _emb_data1
    else:
        emb_data1 = _emb_data1[:-1]
    count1 = KDTree(emb_data1, metric=metric).query_radius(emb_data1, r,
                                                           count_only=True
                                                           ).astype(np.float64)
    # compute phi(order + 1, r)
    emb_data2 = _embed(x, order + 1, 1)
    count2 = KDTree(emb_data2, metric=metric).query_radius(emb_data2, r,
                                                           count_only=True
                                                           ).astype(np.float64)
    if approximate:
        phi[0] = np.mean(np.log(count1 / emb_data1.shape[0]))
        phi[1] = np.mean(np.log(count2 / emb_data2.shape[0]))
    else:
        phi[0] = np.mean((count1 - 1) / (emb_data1.shape[0] - 1))
        phi[1] = np.mean((count2 - 1) / (emb_data2.shape[0] - 1))
    return phi



def _numba_sampen(x, order, r):
    """
    Fast evaluation of the sample entropy using Numba.
    """
    n = x.size
    n1 = n - 1
    order += 1
    order_dbld = 2 * order

    # Define threshold
    # r *= x.std()

    # initialize the lists
    run = [0] * n
    run1 = run[:]
    r1 = [0] * (n * order_dbld)
    a = [0] * order
    b = a[:]
    p = a[:]

    for i in range(n1):
        nj = n1 - i

        for jj in range(nj):
            j = jj + i + 1
            if abs(x[j] - x[i]) < r:
                run[jj] = run1[jj] + 1
                m1 = order if order < run[jj] else run[jj]
                for m in range(m1):
                    a[m] += 1
                    if j < n1:
                        b[m] += 1
            else:
                run[jj] = 0
        for j in range(order_dbld):
            run1[j] = run[j]
            r1[i + n * j] = run[j]
        if nj > order_dbld - 1:
            for j in range(order_dbld, nj):
                run1[j] = run[j]

    m = order - 1

    while m > 0:
        b[m] = b[m - 1]
        m -= 1

    b[0] = n * n1 / 2
    a = np.array([float(aa) for aa in a])
    b = np.array([float(bb) for bb in b])
    p = np.true_divide(a, b)
    #-log(p[-1])
    return -log(p[-1])


def sample_entropy(x, order=2, metric='chebyshev'):
    x = np.asarray(x, dtype=np.float64)
    if metric == 'chebyshev' and x.size < 5000:
        return _numba_sampen(x, order=order, r=(0.2 * x.std(ddof=0)))
    else:
        phi = _app_samp_entropy(x, order=order, metric=metric,
                                approximate=False)
        return -np.log(np.divide(phi[1], phi[0]))


In [ ]:
def se(signal):
    data=[]
    fs=128
    win = 4 * fs
    f, p = welch(signal, fs, nperseg=win)
    # bands
    band = {'delta':[0.5, 4], 'theta':[4, 8], 'alpha':[8, 12], 'beta':[12, 30]}

    # for each band
    try:
      sp_ent = spectral_entropy(signal, sf=128, method='welch')
      data.append(sp_ent)
    except:
      data.append(np.float64(0))
    return data

In [ ]:
def sa(signal):
    fs=128
    data2=[]
    fft_vals = np.fft.rfft(signal)
    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(signal), 1.0/fs)

  # Define EEG bands
    eeg_bands = {'Delta': (0.5, 4),
              'Theta': (4, 8),
              'Alpha': (8, 12),
              'Beta': (12, 30),
              }

    eeg_band_fft = dict()
    freq_ix = np.where((fft_freq >=0) & (fft_freq <=45))[0]
    try:
        sp_ent = sample_entropy(np.array(fft_vals[freq_ix]), order=2)
        data2.append(sp_ent)
    except:
        data2.append(np.float64(0))
    return data2

In [ ]:
def rel_power(eeg):
        sf=128
        freqs, psd = sig.welch(eeg, sf, nperseg=4*sf)
        # Frequency resolution
        freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25

        # Compute the absolute power by approximating the area under the curve
        # Define delta lower and upper limits
        low, high = 1, 4

        # Find intersecting values in frequency vector

        idx_delta = np.logical_and(freqs >= low, freqs <= high)


        low, high = 4, 8
        idx_theta = np.logical_and(freqs >= low, freqs <= high)

        low, high = 8, 13
        idx_alpha = np.logical_and(freqs >= low, freqs <= high)

        low, high = 13, 32
        idx_beta = np.logical_and(freqs >= low, freqs <= high)

        low, high = 32, 64
        idx_gamma = np.logical_and(freqs >= low, freqs <= high)

        try:
          delta_power = simps(psd[idx_delta], dx=freq_res)
        except:
          delta_power=0;
        theta_power = simps(psd[idx_theta], dx=freq_res)

        alpha_power = simps(psd[idx_alpha], dx=freq_res)

        beta_power = simps(psd[idx_beta], dx=freq_res)


        # Relative power (expressed as a percentage of total power)
        try:
          total_power = simps(psd, dx=freq_res)
        except:
          total_power=0
        delta_rel_power = delta_power / total_power
        theta_rel_power = theta_power / total_power

        alpha_rel_power = alpha_power / total_power

        beta_rel_power = beta_power / total_power

        data=[]
        'delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power'
        data.append(delta_rel_power)
        data.append(theta_rel_power)
        data.append(alpha_rel_power)
        data.append(beta_rel_power)
        return data


In [ ]:
def compute_features(eeg_signal):
    # Compute FFT of the signal
    fs=128
    freqs = np.fft.fftfreq(len(eeg_signal), 1/fs)
    fft_vals = np.fft.fft(eeg_signal)

    # Define frequency bands of interest
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 100)

    # Compute PSD in each frequency band
    psd_delta = np.square(np.abs(fft_vals[(freqs >= delta_band[0]) & (freqs <= delta_band[1])])).mean()
    psd_theta = np.square(np.abs(fft_vals[(freqs >= theta_band[0]) & (freqs <= theta_band[1])])).mean()
    psd_alpha = np.square(np.abs(fft_vals[(freqs >= alpha_band[0]) & (freqs <= alpha_band[1])])).mean()
    psd_beta = np.square(np.abs(fft_vals[(freqs >= beta_band[0]) & (freqs <= beta_band[1])])).mean()
    psd_gamma = np.square(np.abs(fft_vals[(freqs >= gamma_band[0]) & (freqs <= gamma_band[1])])).mean()
    theta_beta_ratio = psd_theta / psd_beta
    theta_alpha_ratio = psd_theta / psd_alpha
    alpha_beta_ratio = psd_alpha / psd_beta
    # Calculate spectral entropy
    try:
      spectral_entropy_value = se(eeg_signal)
    except:
      spectral_entropy_value =[]
    # Calculate sample entropy
    try:
      sample_entropy_value = sa(eeg_signal)
    except:
      sample_entropy_value=[]
    rel_power_1=rel_power(eeg_signal)
    # Return PSD in all frequency bands as a list
    ans=[]
    # ans.append[psd_theta, psd_alpha, psd_beta, psd_gamma,theta_beta_ratio,theta_alpha_ratio,alpha_beta_ratio]
    ans.append(psd_delta)
    ans.append(psd_theta)
    ans.append(psd_alpha)
    ans.append(psd_beta)
    ans.append(psd_gamma)
    ans.append(theta_beta_ratio)
    ans.append(theta_alpha_ratio)
    ans.append(alpha_beta_ratio)
    return ans+spectral_entropy_value+sample_entropy_value+rel_power_1

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/SAM40/filtered_data_csv/Arithmetic_task/Arithmetic_sub_10_trial3.csv")
df=df.drop(['Unnamed: 0'], axis=1)
df=df.T
df

,0,1,2,3,4,5,6,7,8,9,...,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199
0,2.180162,1.953459,2.164102,3.146598,-0.739092,4.469622,10.894372,0.025817,-7.455008,1.705766,...,0.343991,0.778640,-1.473768,4.228913,7.262939,-1.561837,-4.415887,2.137563,2.730363,-5.630506
1,4.076474,-1.155726,-1.995117,-0.688743,-1.823242,8.052701,15.224774,2.655199,-6.233611,0.057291,...,-3.554252,-6.492052,-2.611571,4.185851,0.778853,-8.524742,-5.411497,6.811533,8.236696,-1.712062
2,0.628075,-1.537245,-0.033349,-1.492097,-1.853550,9.188221,12.727200,0.005233,-4.895570,-0.283169,...,-4.278173,-6.420603,-3.144149,4.269069,1.707337,-8.207305,-4.826846,8.818823,10.652797,-0.241275
3,0.708820,-1.290217,0.708202,-0.964082,-2.055391,8.649724,12.630578,-0.227815,-7.159756,-3.569116,...,-4.738963,-3.540706,-1.495023,3.095086,2.860028,-5.847177,-5.506619,7.062131,10.443112,0.237470
4,3.604096,-0.514714,0.244670,0.094955,-2.032126,7.623877,12.949440,0.540433,-8.874176,-5.769084,...,-3.092332,0.250636,0.211974,5.270234,5.354965,-6.712716,-7.557503,5.662094,7.364646,-2.982379
5,5.168845,-1.416009,-2.016690,1.616070,-3.170370,3.897624,13.797106,2.374824,-11.457002,-4.837046,...,-1.481719,0.970231,2.017698,6.070303,6.809289,-3.290620,-7.648592,1.707492,4.695289,-4.944100
6,3.104778,0.672418,-0.627448,0.013753,-3.275484,3.043533,11.735279,1.919738,-11.059123,-5.344159,...,-1.314482,1.370050,2.702570,7.202843,6.949939,-2.669363,-5.766458,0.484241,2.271571,-4.316976
7,2.277013,1.784999,-0.581648,-2.836685,-3.788821,3.792214,8.440069,-0.286820,-11.205699,-7.376549,...,-1.952463,0.586964,1.726964,6.489901,7.320710,-2.617924,-6.221348,1.346850,4.086626,-2.362698
8,1.560737,2.823754,0.071229,-3.359821,-2.188388,4.387111,7.029490,-1.044052,-11.960788,-8.793080,...,-1.602181,-0.925386,-0.632386,5.059885,5.776958,-5.216552,-6.494892,2.946209,5.256669,-0.458028
9,1.908722,2.238942,-0.560303,-3.511708,-3.626096,2.862540,6.417737,-1.289378,-10.652401,-7.523400,...,-0.649683,-0.372476,-0.245102,4.524277,5.742065,-2.688772,-5.682686,1.529619,4.514235,-0.850608


In [ ]:
df2=np.array(df['0'])
df3=np.array(df2[384:512])
ans=compute_features(df3)
# for i in range(len(ans)):
#     if math.isinf(ans[i]):
#         ans[i] =np.float64(0)
# print(ans)
for i in range(len(ans)):
  print(i,ans[i],type(ans[i]))

0 4563.68355334319 <class 'numpy.float64'>
1 13962.089350585336 <class 'numpy.float64'>
2 12923.844046750504 <class 'numpy.float64'>
3 5822.533504667666 <class 'numpy.float64'>
4 3073.9622683885527 <class 'numpy.float64'>
5 2.397940576794028 <class 'numpy.float64'>
6 1.0803356416310117 <class 'numpy.float64'>
7 2.219625535239949 <class 'numpy.float64'>
8 4.618853257591647 <class 'numpy.float64'>
9 2.9444389791664407 <class 'float'>
10 0.008957164684743602 <class 'numpy.float64'>
11 0.11768060443483615 <class 'numpy.float64'>
12 0.16465732358902008 <class 'numpy.float64'>
13 0.2606241992842145 <class 'numpy.float64'>


/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)


In [ ]:
def frequency_features(s):
  arr = []
  for i in range(1,32):
      channel = np.asarray(s.iloc[i:i+1, 1:])
      channel = channel.reshape(-1)
      for j in range(0, 25):
          window = channel[j*128:(j+1)*128]
          temp=compute_features(window)
          arr.append(temp)
  channel = np.asarray(s.iloc[31:, 1:])
  channel = channel.reshape(-1)
  for j in range(0, 25):
      window = channel[j*128:(j+1)*128]
      temp=compute_features(window)
      arr.append(temp)
  return arr

In [ ]:
ans=frequency_features(df)
print(ans[0])

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

[14850.830822513803, 14923.21509433007, 7910.065985374859, 6950.560914689403, 3047.5820952902554, 2.1470519109891057, 1.8866106960323739, 1.1380471421605163, 4.911304058644469, 2.639057329615259, 0.05923673299985342, 0.19944375207295562, 0.1836377358760744, 0.24985377337339923]


In [ ]:
s="/content/drive/MyDrive/SAM40/filtered_data/Arithmetic_task/Arithmetic_sub_10_trial1.mat"
data=loadmat(s)
x=data["Clean_data"]
s=pd.DataFrame(x)
arr=frequency_features(s)
ans=np.array(arr)
ans.shape

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

(800, 14)

In [ ]:
def reset(arr):
  ans=[]
  for i in range(25):
    temp=[]
    for j in range(32):
      for z in range(14):
        # print(j*25+i)
        temp.append(arr[j*25+i][z])
    ans.append(temp)
  return ans

In [ ]:
def naming(featureList,CHANNELS):
  ans=[]
  for i in range(32):
    for j in range (14):
      str=CHANNELS[i]+'.'+featureList[j]
      ans.append(str)
  return ans

In [ ]:
s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Arithmetic_task/Arithmetic_sub_11_trial1.csv"
df = pd.read_csv(s)
df=df.drop(['Unnamed: 0'], axis=1)
df=df.T
s=pd.DataFrame(df)
arr=frequency_features(s)
ans=np.array(arr)
ans=reset(ans)
featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
headerList = naming(featureList,CHANNELS)
s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Frequency_domain_features/Arithmetic_task/Arithmetic_sub_11_trial1.csv"
DF = pd.DataFrame(ans)
# headerList = ['mean_value','variance_value','std_value','min_value','max_value','peak_to_peak_value','median','var','hj_mb','hj_comp']
# file.to_csv("gfg2.csv", header=headerList, index=False)
# ,header=headerList
DF.to_csv(s,header=headerList)
DF

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-4-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = 

,0,1,2,3,4,5,6,7,8,9,...,438,439,440,441,442,443,444,445,446,447
0,31685.028065,22677.984555,14422.435207,11717.368463,5514.631070,1.935416,1.572410,1.230860,4.873917,1.945910,...,9139.528497,1.934016,2.234714,0.865442,4.575953,2.079442,0.304499,0.136386,0.091296,0.403584
1,27429.310211,37782.925666,40291.449432,10737.957004,4549.486910,3.518633,0.937741,3.752245,4.618566,0.916291,...,10680.942456,4.306046,2.841926,1.515186,4.402704,2.639057,0.061540,0.423149,0.093318,0.278337
2,31694.541328,27268.830968,9632.061907,17779.517547,8553.959510,1.533722,2.831048,0.541750,4.934552,0.000000,...,17367.989653,2.639337,4.322155,0.610653,4.883693,1.757858,0.106452,0.150112,0.119417,0.360443
3,21590.687781,18971.687148,12582.630332,13836.282705,7657.660537,1.371155,1.507768,0.909394,4.909300,1.386294,...,14334.785040,1.503868,1.381303,1.088731,4.586304,0.000000,0.232500,0.171196,0.103603,0.209250
4,38212.139186,16007.353294,6495.931393,13160.961671,6006.755932,1.216275,2.464212,0.493576,4.485203,1.871802,...,12908.854273,1.939966,3.104604,0.624868,4.484413,0.000000,0.220416,0.215615,0.035351,0.328114
5,19103.302145,41939.994597,20445.611512,10803.412828,4829.838286,3.882106,2.051296,1.892514,4.693086,1.704748,...,5831.349123,5.123355,2.523999,2.029856,4.643217,2.397895,0.175992,0.283703,0.135524,0.349392
6,40424.767936,30166.208220,14130.656209,10455.190199,2728.055890,2.885285,2.134806,1.351545,4.669677,0.000000,...,8431.919816,3.074800,2.652558,1.159183,4.475602,2.995732,0.356238,0.099055,0.136096,0.349182
7,20591.626588,20492.666833,13580.691309,10694.132332,4580.769000,1.916253,1.508956,1.269920,4.705187,0.000000,...,7650.735012,2.251398,1.112637,2.023479,4.644938,0.000000,0.136834,0.117120,0.217132,0.390190
8,22855.909960,23872.459411,27507.008168,8376.864821,5092.842638,2.849808,0.867868,3.283688,4.810601,0.000000,...,4702.897342,3.039476,1.324541,2.294738,4.681469,2.397895,0.122527,0.293341,0.222846,0.284256
9,28118.076237,36961.971939,15093.204128,4461.865184,2804.238076,8.283973,2.448915,3.382712,4.418042,2.014903,...,5699.073340,3.968617,3.686186,1.076619,4.808881,2.484907,0.102193,0.316413,0.120460,0.274720


In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/filtered_data/Mirror_image_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/filtered_data/Mirror_image_task/" + your_file_name
  data=loadmat(s)
  x=data["Clean_data"]
  s=pd.DataFrame(x)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Features_filtered_data/Frequency_domain_features/Mirror_image/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)


In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/filtered_data/Arithmetic_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/filtered_data/Arithmetic_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Features_filtered_data/Frequency_domain_features/Arithmetic_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/filtered_data/Relax")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/filtered_data/Relax/" + your_file_name
  data=loadmat(s)
  x=data["Clean_data"]
  s=pd.DataFrame(x)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Features_filtered_data/Frequency_domain_features/Relax/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)


In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/filtered_data/stroop_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/filtered_data/stroop_task/" + your_file_name
  data=loadmat(s)
  x=data["Clean_data"]
  s=pd.DataFrame(x)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Features_filtered_data/Frequency_domain_features/stroop_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)


In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Arithmetic_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Arithmetic_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data_features/Frequency_domain_features/Arithmetic_features/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Mirror_image_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Mirror_image_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data_features/Frequency_domain_features/Mirror_image_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Relax")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Relax/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data_features/Frequency_domain_features/Relax/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Stroop_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data/Stroop_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_data_features/Frequency_domain_features/Stroop_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Arithmetic_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Arithmetic_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Frequency_domain_features/Arithmetic_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Mirror_image_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Mirror_image_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Frequency_domain_features/Mirror_image_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Relax")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Relax/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Frequency_domain_features/Relax/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 

In [ ]:
list_of_files=os.listdir("/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Stroop_task")
for your_file_name in list_of_files:
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data/Stroop_task/" + your_file_name
  df = pd.read_csv(s)
  df=df.drop(['Unnamed: 0'], axis=1)
  df=df.T
  s=pd.DataFrame(df)
  arr=frequency_features(s)
  ans=np.array(arr)
  ans=reset(ans)
  featureList=['psd_delta','psd_theta', 'psd_alpha', 'psd_beta', 'psd_gamma','theta_beta_ratio','theta_alpha_ratio','alpha_beta_ratio','spectral_entropy_value','sample_entropy_value','delta_rel_power','theta_rel_power','alpha_rel_power','beta_rel_power']
  CHANNELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
  headerList = naming(featureList,CHANNELS)
  size = len(your_file_name)
  new_name = your_file_name[:size - 4]
  s="/content/drive/MyDrive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Frequency_domain_features/Stroop_task/"+new_name+"frequencyFeat"+".csv"
  DF = pd.DataFrame(ans)
  # file.to_csv("gfg2.csv", header=headerList, index=False)
  DF.to_csv(s,header=headerList)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
<ipython-input-201-a8fee32232d6>:122: ComplexWarning: Casting complex values to real discards the imaginary part
  x = np.asarray(x, dtype=np.float64)
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 512 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 127, using nperseg = 127
  warnings.warn('nperseg 